In [ ]:
import pandas as pd #module to work with dataframes
import networkx as nx #module to work with networks
import numpy as np
import matplotlib.pyplot as plt
from Functions import *

# Dynamics on networks and dynamics of networks

# Dynamics of networks: Temporal networks

To get some practice with temporal networks we will analyze the co-occurrence network of the characters in the Game of Thrones books. Here, two characters are considered to co-occur if their names appear in the vicinity of 15 words from one another in the books. The edge weight corresponds to the number of interactions. . The dataset is publicly avaiable for the 5 books at https://github.com/mathbeveridge/asoiaf. Let's take a look at it

In [ ]:
books = load_game_of_thrones_data()

In [ ]:
books

As you can see this data easily translates to a network problem. Now it's time to create a network.
We create a graph for each book. It's possible to create one `MultiGraph`(Graph with multiple edges between nodes) instead of 5 graphs, but it is easier to analyse and manipulate individual `Graph` objects rather than a `MultiGraph`. We can consider this a sort of **temporal network**, where each book is a different snapshot at the relationshiops between characters. As we have  books we will have 5 snapshots. While there are librars that allow you to better work with temporal networks, you can still do that by combining your analysis of the 5 networks.

In [ ]:
# we create a list of graph objects using
# nx.from_pandas_edgelist and specifying
# the edge attributes.

graphs = [nx.from_pandas_edgelist(
          books[books.book==i],
          source='Source', target='Target',
          edge_attr=['weight', 'weight_inv'])
          for i in range(1, 6)]

In [ ]:
#what is contained in the list? # a list of 5 networks
graphs

In [ ]:
# The Graph object associated with the first book.
graphs[0]

In [ ]:
# To access the relationship edges in the graph use(data=True)
relationships = list(graphs[0].edges(data=True)) #creates a list of all the edges with their attributes (only weight, as "book" we took it out when we did 5 networks!!

In [ ]:
relationships[0:3]

## Finding the most important node i.e character in these networks.

Let's use our network analysis knowledge to see what we can learn of these Graphs that we have just created, and their changes trough time.

Is it Jon Snow, Tyrion, Daenerys, or someone else the most central character? Let's see! Network Science offers us many different metrics to measure the importance of a node in a network as we saw in lesson 2A. Note that there is no "correct" way of calculating the most important node in a network, every metric has a different meaning.

First, let's measure the importance of a node in a network by looking at the number of neighbors it has, that is, the number of nodes it is connected to. 

<div class="alert alert-block alert-success"><b>Up to you:</b>
<h4> Exercise 24</h4>
    
Considering that you seek to compare the importance of same nodes across different networks (as the network will be changing in time) in term of their **number of neighbours**, what metric should you use, **degree cenrtality** or **node degree**?
    
    
</div>

In [ ]:
# no need to code, just think and remember!
# %load ./snippets/ex24.py

Using this measure, let's extract the top ten important characters from the first book (`graphs[0]`) and the fifth book (`graphs[4]`).

In [ ]:
# We use the in-built degree_centrality method
deg_cen_book1 = pd.Series(dict(nx.degree_centrality(graphs[0]))).sort_values(ascending=False)
deg_cen_book5 = pd.Series(dict(nx.degree_centrality(graphs[4]))).sort_values(ascending=False)

In [ ]:
print(deg_cen_book1.head(10))
print(deg_cen_book5.head(10))

## Evolution of importance of characters over the books

According to degree centrality the most important character in the first book is Eddard Stark but he is not even in the top 10 of the fifth book. 
The importance changes over the course of five books, because you know stuff happens ;)

How can we study changes in characters cenrtality? For example looking at the evolution of degree centrality of some relevant charactesr like Eddard Stark, Jon Snow, or Tyrion which showed up in the top 10 of degree centrality in first book.

For that, we need to create a dataframe that contains: the character, the book, and the character importance in that book. We will create a dataframe with character columns and index as books, where every entry is the degree centrality of the character in that particular book and plot the evolution of the degree centrality of Eddard Stark, Jon Snow and Tyrion.

In [ ]:
evolK = [nx.degree_centrality(graph) for graph in graphs] #creates a list containing the dictionaries of degree centralities of the charactesr
evolK_df = pd.DataFrame.from_records(evolK).fillna(0) #create a dataframe using book as index, characters as columns, and centrality as values

In [ ]:
evolK_df.head()

In [ ]:
characters=['Eddard-Stark','Tyrion-Lannister','Jon-Snow']#'Daenerys-Targaryen','Arya-Stark'
evolK_df[characters].plot(style='o-')
plt.ylabel("Character centrality")
plt.xlabel("Book")
plt.show()

We can see that the importance of Eddard Stark in the network dies off and with Jon Snow there is a drop in the fourth book but a sudden rise in the fifth book

## Betweeness centrality

Let's do this for Betweeness centrality and check if this makes any difference. As different centrality method use different measures underneath, they find nodes which are important in the network. A centrality method like Betweeness centrality finds nodes which are structurally important to the network, which binds the network together and densely.

In [ ]:
evolB = [nx.betweenness_centrality(graph, weight="weight_inv") for graph in graphs]
evolB_df = pd.DataFrame.from_records(evolB).fillna(0)

In [ ]:
#let's find the characters that are more relevant por joining many plots:
characters=evolB_df.sum().sort_values(ascending=False).index.to_list()[0:5]

In [ ]:
evolB_df[characters].plot(style='o-')
plt.ylabel("Character betweennes")
plt.xlabel("Book")
plt.show()

## Evolution of Roles over time
We can now determine characters that are incresing or decreasing in importance in the plot of the books!

In [ ]:
all_characters=evolK_df.columns
evolK_df.reset_index(names="book",inplace=True)

In [ ]:
Centrality_evol=evolK_df[all_characters].corrwith(evolK_df["book"])

In [ ]:
loosers=Centrality_evol.sort_values(ascending=True).index.to_list()[0:3]
winners=Centrality_evol.sort_values(ascending=False).index.to_list()[0:3]

In [ ]:
Centrality_evol.sort_values(ascending=False)

In [ ]:
evolK_df[loosers + winners ].plot(style='o-')

## Node and Link constancy

In temporal networks we can also see if nodes and links persist over time. We can maka a bipartite "network" of books and nodes, and books and edges, and easily see their presence trough time

<div class="alert alert-block alert-success"><b>Up to you: </b>
<h4> Exercise 25</h4>
    
Create a bipartite network of books and interactions. In that network the nodes in one set are the different books, while the nodes in the second set are the interactions between the characters.
These types of networks are used, for example, to study how ecological interactions are structured in different fields.
    
- 1.  How many interactions characters are shared between the first and the second book?
- 2.  What books share the most number of character interactions?
    
Create a bipartite network of books and characters. In that network the nodes in one set are the different books, while the nodes in the second set are the characters.
- 3. What books share the most number of characters?
</div>

In [ ]:
## your code here


In [ ]:
# %load ./snippets/ex25.py

In [ ]:
# %load ./snippets/ex25b.py

## Air traffic
we will analyse the evolution of US Airport Network between 1990 and 2015. This dataset contains data for 25 years[1995-2015] of flights between various US airports and metadata about these routes. Taken from Bureau of Transportation Statistics, United States Department of Transportati

In [ ]:
def load_airports_data():
    pass_air_data = pd.read_csv("./data/passengers.csv", index_col="id")
    return pass_air_data

In [ ]:
pass_air_data=load_airports_data()

In [ ]:
pass_air_data.head()

In [ ]:
passenger_graph = nx.from_pandas_edgelist(
    pass_air_data, source='ORIGIN',
    target='DEST', edge_key='YEAR',
    edge_attr=['PASSENGERS', 'UNIQUE_CARRIER_NAME'],
    create_using=nx.MultiDiGraph())

We have created a MultiDiGraph object `passenger_graph` which contains all the information from the dataframe `pass_air_data`. `ORIGIN` and `DEST` represent the column names in the dataframe `pass_air_data` used to construct the edge. As this is a `MultiDiGraph` we can also give a name/key to the multiple edges between two nodes and `edge_key` is used to represent that name and in this graph `YEAR` is used to distinguish between multiple edges between two nodes. `PASSENGERS` and `UNIQUE_CARRIER_NAME` are added as edge attributes which can be accessed using the nodes and the key form the MultiDiGraph object.

Let's check if can access the same information (the 2006 route between JFK and AUS) using our `passenger_graph`.

To check an edge between two nodes in a Graph we can use the syntax `GraphObject[source][target]` and further specify the edge attribute using `GraphObject[source][target][attribute]`.

<!-- Let's see if `passenger_graph['JFK']['AUS'][2006]` works. -->

In [ ]:
passenger_graph['JFK']['AUS'][2006]

In [ ]:
# Route betweeen New York-JFK and SFO

values = [(year, attr['PASSENGERS'])
          for year, attr in 
          passenger_graph['JFK']['SFO'].items()]
x, y = zip(*values)
plt.plot(x, y)
plt.show()

In [ ]:
pass_air_data.groupby(
    ['YEAR']).sum()['PASSENGERS'].plot()
plt.show()

In [ ]:
def year_network(G, year):
    """ Extract edges for a particular year from
        a MultiGraph. The edge is also populated with
        two attributes, weight and weight_inv where
        weight is the number of passengers and
        weight_inv the inverse of it.
    """
    year_network = nx.DiGraph()
    for edge in G.edges:
        source, target, edge_year = edge
        if edge_year == year:
            attr = G[source][target][edge_year]
            year_network.add_edge(
                source, target,
                weight=attr['PASSENGERS'],
                weight_inv=1/(attr['PASSENGERS']
                if attr['PASSENGERS'] != 0.0 else 1),
                airlines=attr['UNIQUE_CARRIER_NAME'])
    return year_network

In [ ]:
pass_2015_network = year_network(passenger_graph, 2015)

In [ ]:
pass_2015_network